In [ ]:
import os
import openai
import pandas as pd
import random
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
openai.api_key = 'API KEY'

In [50]:
file_path1 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling bengali\urls_and_articles_summary_bengali.csv"
file_path2 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling gujarati\urls_and_articles_summary_gujarati.csv"
file_path3 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling hindi\urls_and_articles_summary_hindi.csv"
file_path4 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling marathi\urls_and_articles_summary_marathi.csv"
file_path5 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling tamil\urls_and_articles_summary_tamil.csv"
file_path6 = r"D:\Machine translation and summarization model\Summerization model\Random samples of test data splits\randomsampling telugu\urls_and_articles_summary_telugu.csv"

# Load each file and store in a separate DataFrame
df1 = pd.read_csv(file_path1, encoding='utf-8')
df2 = pd.read_csv(file_path2, encoding='utf-8')
df3 = pd.read_csv(file_path3, encoding='utf-8')
df4 = pd.read_csv(file_path4, encoding='utf-8')
df5 = pd.read_csv(file_path5, encoding='utf-8')
df6 = pd.read_csv(file_path6, encoding='utf-8')

# Preview the data for each file to confirm the "Article" column is loaded correctly
print("File 1 Preview:")
print(df1.head())

print("File 2 Preview:")
print(df2.head())

print("File 3 Preview:")
print(df3.head())

print("File 4 Preview:")
print(df4.head())

print("File 5 Preview:")
print(df5.head())

print("File 6 Preview:")
print(df6.head())

File 1 Preview:
                                  URL Source  \
0  https://www.bbc.com/bengali/news-56713767   
1  https://www.bbc.com/bengali/news-56533637   
2  https://www.bbc.com/bengali/news-56631353   
3  https://www.bbc.com/bengali/news-47189627   
4  https://www.bbc.com/bengali/news-43949363   

                                             Article  \
0  বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসল...   
1  বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুল...   
2  ডুবে যাওয়া লঞ্চটি মুন্সিগঞ্জ থেকে নারায়ণগঞ্জ...   
3  ২০১৮ সালের নির্বাচন প্রত্যাখ্যান করেছে বিএনপি।...   
4  কার্টুনে দাম্পত্য জীবন মহারাষ্ট্রের কোলাপুরের ...   

                                             Summary  
0  ইসলামী শিক্ষাবিদ এবং যুক্তরাজ্যের জাতীয় স্বাস...  
1  বাংলাদেশে ভারতীয় প্রধানমন্ত্রী নরেন্দ্র মোদীর...  
2  বাংলাদেশে রাজধানী ঢাকার নিকটবর্তী নারায়ণগঞ্জে...  
3  বাংলাদেশে গত ৩০শে ডিসেম্বর অনুষ্ঠিত সংসদ নির্ব...  
4  বিয়ে টিকিয়ে রাখার ক্ষেত্রে স্বামী ও স্ত্রীর ...  
File 2 Preview:
            

In [64]:
def get_last_full_stop(text):
    """Helper function to find the last full stop or '।' in the summary."""
    last_full_stop = max(text.rfind('.'), text.rfind('।'))
    if last_full_stop != -1:
        return text[:last_full_stop + 1]
    return text  

def summarize_article(article, target_language, max_tokens=100):
    try:
        # Create the prompt for the OpenAI model
        prompt = f"Summarize the following article in {target_language}. Please ensure the summary is complete, concise, and includes all the key points.\n\nArticle:\n{article}\n\nSummary:"

        # Make an API request to OpenAI's GPT model
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Using the GPT-4.0 Mini model
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,  # Limit the number of tokens for the response
            temperature=random.uniform(0.6, 1.0),  # Add variability in the response
            n=1  # Number of responses to generate
        )

        # Extract the summary text from the response
        summary = response['choices'][0]['message']['content'].strip()

        # Post-process the summary to ensure it ends at the last full stop or '।'
        summary = get_last_full_stop(summary)

        return summary

    except Exception as e:
        print(f"Error summarizing article: {e}")
        return None

In [66]:
def process_dataframe(df, target_language):
    all_summaries = []

    # Iterate over each article in the DataFrame
    for index, article in enumerate(df['Article']):
        summary = summarize_article(article, target_language)
        all_summaries.append(summary)

        # Print the generated summary for the current article in real-time
        print(f"Processing Article {index + 1}/{len(df)}:")
        print(f"Article: {article[:100]}...")  # Print the first 100 characters of the article for context
        print(f"Generated Summary: {summary}")
        print("-" * 100)  # Separator for clarity

    df['Generated_Summary'] = all_summaries
    return df

In [68]:
target_language = "Hindi"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df3,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\GPT\GPT_summaries_Hindi.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: दोनों ने भारत के वीनू मनकड और पंकज राय का 52 साल पुराना रिकॉर्ड तोड़ दिया. स्मिथ और मैकेंज़ी ने चटगा...
Generated Summary: दक्षिण अफ्रीकी क्रिकेटर्स स्मिथ और मैकेंज़ी ने भारत के वीनू मनकड और पंकज राय का 52 साल पुराना रिकॉर्ड तोड़ दिया। उन्होंने बांग्लादेश के खिलाफ दूसरे टेस्ट के दूसरे दिन पहले विकेट के लिए 415 रन बनाकर नया विश्व रिकॉर्ड स्थापित किया। मनकड और राय ने 1956 में 413 रन की साझेदारी की थी।
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: ये खबर इंडियन एक्सप्रेस के पहले पन्ने पर है. पार्टी ने कहा कि अगर वो पश्चिम बंगाल में जीतती है तो वह...
Generated Summary: यह लेख इंडियन एक्सप्रेस के पहले पन्ने पर छपी कुछ प्रमुख खबरों का विवरण देता है। भारतीय जनता पार्टी (बीजेपी) ने कहा है कि यदि वह पश्चिम बंगाल में जीतती है, तो वहां राष्ट्रीय नागरिकता रजिस्टर (NRC) तैयार करेगी। बीजेपी के प्रभारी कैलाश विजयवर्गीय ने दावा किय

In [70]:
target_language = "Bengali"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df1,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\GPT\GPT_summaries_Bengali.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: বাংলাদেশের ইসলামিক ফাউন্ডেশনও গত ১৪ই মার্চ ইসলামিক ফাউন্ডেশনের সভা কক্ষে দেশের জ্যেষ্ঠ আলেমদের সঙ্গে...
Generated Summary: বাংলাদেশের ইসলামিক ফাউন্ডেশন ১৪ই মার্চ রোজা রেখে করোনাভাইরাসের টিকা নেওয়ার বিষয়ে আলেমদের সঙ্গে আলোচনা করে জানিয়েছে যে, রোজাদাররা দিনের বেলায় টিকা নিতে পারেন, কারণ টিকা মাংসপেশিতে দেওয়া হয় এবং এটি খাদ্যনালীতে প্রবেশ করে না।
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: বিক্ষোভের সময় পুলিশ কাঁদানে গ্যাস ও রাবার বুলেট ছোঁড়ে, বিক্ষোভকারীরাও ইটপাটকেল নিক্ষেপ করে। জানা গ...
Generated Summary: শুক্রবার বায়তুল মোকাররমের সামনে বিক্ষোভের সময় সংঘর্ষের ঘটনা ঘটে। জুমার নামাজের পর মোদী-বিরোধী শ্লোগান দিয়ে মিছিল শুরু হলে পুলিশ ওই বিক্ষোভে বাধা দেয়, ফলে সংঘর্ষের সূত্রপাত হয়।
----------------------------------------------------------------------------------------------------
Processing Article 3/150

In [72]:
target_language = "Gujarati"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df2,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\GPT\GPT_summaries_Gujarati.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: જોકે, રાજપૂતાના રાઇફલ્સની બીજી બટાલિયનમાં જોડાયેલા લેફ્ટનન્ટ હિતેશ કુમારના પિતા લાન્સ નાયક બચન સિંહ ...
Generated Summary: લેફ્ટનન્ટ હિતેશ કુમાર, રાજપૂતાના રાઈફલ્સની બીજી બટાલિયનમાં જોડાયેલા છે, પરંતુ તેમના પિતા, લાન્સ નાયક બચન સિંહ, કારગિલ યુદ્ધમાં 12 જૂન, 1999ના રોજ શહીદ થઈ ગયા હતા.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: સામાચાર એજન્સી એએનઆઈ પ્રમાણે આગ લાગવાની આ ઘટના શૉર્ટ સર્કિટના કારણે થઈ છે. જોકે, અધિકારીઓના જણાવ્યા ...
Generated Summary: આર્ટિકલના અનુસાર, આગ લાગવાની ઘટના શૉર્ટ સર્કિટના કારણે બની હતી, પરંતુ અધિકારીઓએ આગ પર કાબુ મેળવી લીધો છે અને યાત્રીઓ સુરક્ષિત છે. અમદાવાદમાં કોરોના વાઇરસના વધતા કેસોને ધ્યાનમાં રાખીને, રાજપથ અને કર્ણાવતી ક્લબોએ હોળીની ઉજવણી ન કરવાની જાહેરાત કરી છે.
----------------------------------------------------------------------------------------------------
Processing Article 3/1

In [76]:
target_language = "Marathi"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df4,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\GPT\GPT_summaries_Marathi.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: लवकर केस पांढरे होत असतील तर त्याला वैद्यकीय भाषेत याला केनाइटिस असं म्हणतात. वयाच्या 20 वर्षांआधीचं...
Generated Summary: केस लवकर पांढरे होणे म्हणजेच 'केनाइटिस'. जर वयाच्या 20 वर्षांपूर्वीच तुमचे केस पांढरे झाले असतील, तर तुम्हाला हे असू शकते. यामुळे केस काळे करणारे रंगद्रव्यं कमी होतात.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: स्टीव्हन स्मिथने वेगवान शतक झळकावलं. सिडनी क्रिकेट ग्राऊंडवर सुरू असलेल्या या मॅचमध्ये कोरोना कारणास...
Generated Summary: सिडनी क्रिकेट ग्राऊंडवर झालेल्या वनडे सामन्यात स्टीव्हन स्मिथने वेगवान शतक झळकावले. या सामन्यात कोरोना कारणास्तव मर्यादित प्रेक्षकांना प्रवेश देण्यात आला.
----------------------------------------------------------------------------------------------------
Processing Article 3/150:
Article: कोरोना लसीकरण खुद्द पंतप्रधान मोदींनीही 1 मार्चला, म्हणजे दुसऱ्या टप्प्याच्य

In [77]:
target_language = "Tamil"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df5,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\GPT\GPT_summaries_Tamil.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: தீர்ப்பு வெளியான பிறகு செய்தியாளர்களிடம் பேசிய மு.க. ஸ்டாலின், "ஜனநாயகத்தை காக்கக்கூடிய வகையில் இந்த...
Generated Summary: மு.க. ஸ்டாலின், தி.மு.க.க்கு சாதகமாக உச்ச நீதிமன்றத்தில் வந்த புதிய தீர்ப்பு குறித்து பேசினார். அவர், இந்த தீர்ப்பு ஜனநாயகத்தை காக்கக்கூடியதாக உள்ளது என்று கூறின அவர், உள்ளாட்சித்தேர்தலை நிறுத்துவதற்காக தி.மு.க. நீதிமன்றத்தை நாடவில்லை என்றார்.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: சித்தரிப்புக்காக வெனிசுவேலா நாட்டில் ஏறத்தாழ 50 ஆயிரம் வாக்கு இயந்திரங்கள் எரிந்து சாம்பலானதாக அந்நா...
Generated Summary: வெனிசுவேலாவில் 50,000 வாக்கு இயந்திரங்கள் மற்றும் 582 கணினிகள் தீக்கிரையானதாக தேர்தல் ஆணையம் தெரிவித்துள்ளது. இந்த தீ விபத்து கராகஸ் அருகே உள்ள கிடங்கில் ஏற்பட்டது, ஆனால் தீவிபத்திற்கான காரணம் இன்னும் தெரியவில்லை. இதனால், இந்த ஆண்டு இறுதியில் நடைபெறவிருந்த தேர்தல் தள்ளி போகுமா என்பது பற்றிய

In [78]:
target_language = "Telugu"
print("Processing articles and generating summaries...")
processed_df = process_dataframe(df6,target_language)
output_file_path = "D:\Machine translation and summarization model\Summerization model\OUTPUT SUMMARIES\GPT\GPT_summaries_Telugu.csv" 
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
processed_df.to_csv(output_file_path, index=False, encoding='utf-8-sig') 
print(f"Results saved to {output_file_path}")

Processing articles and generating summaries...
Processing Article 1/150:
Article: ప్రధాని నరేంద్ర మోదీ ప్రైమ్ మినిస్టర్స్ సిటిజన్ అసిస్టెన్స్ అండ్ రిలీఫ్ ఇన్ ఎమర్జెన్సీ సిచువేషన్(పీఎ...
Generated Summary: ప్రధాని నరేంద్ర మోదీ ప్రైమ్ మినిస్టర్స్ సిటిజన్ అసిస్టెన్స్ అండ్ రిలీఫ్ ఇన్ ఎమర్జెన్సీ సిచువేషన్ (పీఎం కేర్స్) ఫండ్‌ను నేషనల్ డిజాస్టర్ రిలీఫ్ ఫండ్ (ఎన్‌డీఆర్ఎఫ్)లోకి బదిలీ చేయాల్సిన అవసరం లేదని భారత సుప్రీంకోర్టు ప్రకటించింది.
----------------------------------------------------------------------------------------------------
Processing Article 2/150:
Article: 2020 అధ్యక్ష ఎన్నికల్లో జోక్యం చేసుకోవడానికి ట్విటర్ ప్రయత్నిస్తోందని ట్రంప్ ఆరోపించారు ''దేర్‌ ఈజ్‌...
Generated Summary: ట్విటర్, 2020 అధ్యక్ష ఎన్నికల్లో డొనాల్డ్ ట్రంప్ చేసిన వాస్తవం కాని ఆరోపణలపై చర్య తీసుకుంటూ, ఆయన మెయిల్-ఇన్‌ బ్యాలట్స్‌కు సంబంధించిన పోస్ట్‌కు హెచ్చరికలేబుల్‌ను జోడించింది. ట్రంప్, "సరైన సమాచారం పంచేందుకు" ట్విటర్ ప్రయత్నిస్తున్నదని ఆరోపించారు.
--------------------------------------------------------------